In [5]:
import requests
from citipy import citipy
import pandas as pd
import numpy as np
from config import openweather_api_key
from datetime import datetime
url = "http://api.openweathermap.org/data/2.5/weather?" + "appid=" + openweather_api_key

In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=80)
lngs = np.random.uniform(low=-180.000, high=180.000, size=80)
coordinates = zip(lats, lngs)

# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.  If greater than
# 60, reduce to 60 to avoid violating the API subscription
if len(cities) > 60:
    cities = cities[:60]
len(cities)

60

In [3]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

    # If an error occurs, skip the city.
    except:
        print(city + " not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")


Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | yellowknife
Processing Record 2 of Set 1 | turukhansk
Processing Record 3 of Set 1 | kapaa
Processing Record 4 of Set 1 | mataura
Processing Record 5 of Set 1 | provideniya
Processing Record 6 of Set 1 | avarua
Processing Record 7 of Set 1 | hadejia
Processing Record 8 of Set 1 | mae hong son
Processing Record 9 of Set 1 | ushuaia
Processing Record 10 of Set 1 | castro
Processing Record 11 of Set 1 | mangan
Processing Record 12 of Set 1 | ajdabiya
Processing Record 13 of Set 1 | chuy
Processing Record 14 of Set 1 | arraial do cabo
Processing Record 15 of Set 1 | tanabe
Processing Record 16 of Set 1 | cabinda
Processing Record 17 of Set 1 | punta arenas
Processing Record 18 of Set 1 | busselton
Processing Record 19 of Set 1 | rikitea
Processing Record 20 of Set 1 | vaitupu
vaitupu not found. Skipping...
Processing Record 21 of Set 1 | piracaia
Processing Record 22 of Set 1 | lorengau
Processing Re

In [8]:
city_data_df = pd.DataFrame(city_data)
city_data_column_order = ['City', 'Country', 'Date', 'Lat', 'Lng', 'Max Temp', 
                          'Humidity', 'Cloudiness', 'Wind Speed']
city_data_df = city_data_df[city_data_column_order]
city_data_df.head(10)

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Yellowknife,CA,2021-09-25 15:29:56,62.4560,-114.3525,280.60,100,90,3.60
1,Turukhansk,RU,2021-09-25 15:34:03,65.8167,87.9833,273.05,83,3,3.52
2,Kapaa,US,2021-09-25 15:34:04,22.0752,-159.3190,297.33,80,1,0.45
3,Mataura,NZ,2021-09-25 15:34:04,-46.1927,168.8643,277.30,97,81,2.27
4,Provideniya,RU,2021-09-25 15:34:04,64.3833,-173.3000,272.05,83,100,3.70
5,Avarua,CK,2021-09-25 15:29:58,-21.2078,-159.7750,298.18,69,90,6.17
6,Hadejia,NG,2021-09-25 15:34:05,12.4498,10.0444,307.15,32,55,2.84
7,Mae Hong Son,TH,2021-09-25 15:34:05,19.3003,97.9685,296.62,98,99,0.81
8,Ushuaia,AR,2021-09-25 15:30:12,-54.8000,-68.3000,278.96,65,75,6.69
9,Castro,BR,2021-09-25 15:34:06,-24.7911,-50.0119,299.24,44,19,3.15


In [10]:
# Save the city data to a CSV
output_data_file = "weather_data/cities.csv"
city_data_df.to_csv(output_data_file, index_label="City_ID")